In [57]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import acquire as a
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
import requests
import unicodedata
import nltk
from textblob import TextBlob, Word
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
import prepare as p
import acquire as a
import new_lib as nl
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = p.get_links('links2.csv')

In [3]:
df2 = a.get_dataframe(df.href)

In [4]:
df2 = p.get_languages(df2)

In [5]:
df2 = p.clean_languages(df2)

In [6]:
df2 = p.drop_blanks(df2)

In [7]:
df2['readme_contents'] = df2['readme_contents'].apply(p.basic_clean)

In [8]:
df2['readme_contents'] = df2['readme_contents'].apply(p.remove_stopwords)

In [9]:
df2['readme_contents'] = df2['readme_contents'].apply(p.tokenized)

In [10]:
df2['readme_contents'] = df2['readme_contents'].apply(p.split)

In [11]:
df2 = df2.dropna()

In [12]:
train, val, test, X_train, y_train, X_val, y_val, X_test, y_test = nl.train_vailidate_test_split(df2, 'language')

In [13]:
len(train)

212

In [14]:
len(val)

92

In [15]:
len(test)

76

In [16]:
train.isna().sum()

repo               0
language           0
readme_contents    0
dtype: int64

In [17]:
train

,repo,language,readme_contents
213,learn-co-students/simple-blackjack-cli-seattle...,Ruby,simplifi blackjack procedur learn goal util co...
21,kevinleedrum/vlackjack,Other,vlackjack spade heart club diamond rocket play...
316,learn-co-students/cli-applications-simple-blac...,Ruby,simplifi blackjack procedur learn goal util co...
71,Shubhamsharma09/PYGames,Python,pygam built python 3httpsimgshieldsiobadgepyth...
374,cs361-W16/Blackjack-21,Java,welcom group21 blackjack build statushttpstrav...
...,...,...,...
341,learn-co-students/cli-applications-simple-blac...,Ruby,simplifi blackjack procedur learn goal util co...
189,learn-co-students/cli-applications-simple-blac...,Ruby,simplifi blackjack procedur learn goal util co...
285,AltspaceVR/mre-blackjack,Other,mre blackjack mre blackjack virtual realiti ga...
94,learn-co-students/cli-applications-simple-blac...,Ruby,simplifi blackjack procedur learn goal util co...


In [18]:
word_list = []
for n in train['readme_contents']:
    word_list.extend(n.split(' '))

In [19]:
series = pd.Series(word_list)

In [20]:
series.value_counts().head(20)

method     9674
card       3435
test       2455
use        2261
line       2068
put        2040
user       1980
command    1978
input      1923
call       1884
player     1826
file       1808
program    1598
output     1591
runner     1573
hit        1459
total      1364
rubi       1346
game       1326
run        1281
dtype: int64

In [21]:
tfidf = TfidfVectorizer()

In [22]:
X_train = tfidf.fit_transform(X_train['readme_contents'])

In [23]:
X_val = tfidf.transform(X_val['readme_contents'])

In [24]:
X_val = pd.DataFrame(X_val.todense())

In [90]:
clf = DecisionTreeClassifier(max_depth=6, random_state=77)

In [91]:
X_train = pd.DataFrame(X_train.todense())

AttributeError: 'DataFrame' object has no attribute 'todense'

In [92]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, random_state=77)

In [93]:
y_pred = clf.predict(X_train)

In [94]:
y_pred_proba = clf.predict_proba(X_train)

In [95]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.86


In [96]:
confusion_matrix(y_train, y_pred)

array([[  6,   0,   0,   4,   0,   0],
       [  0,  13,   0,   8,   0,   0],
       [  0,   0,   7,   9,   0,   0],
       [  0,   0,   0,  23,   0,   0],
       [  0,   0,   0,   8,  19,   0],
       [  0,   0,   0,   1,   0, 114]])

In [97]:
ConfusionMatrixDisplay(confusion_matrix(y_train, y_pred))

In [98]:
pd.crosstab(y_train, y_pred)

col_0,C_based,Java,JavaScript,Other,Python,Ruby
language,,,,,,
C_based,6,0,0,4,0,0
Java,0,13,0,8,0,0
JavaScript,0,0,7,9,0,0
Other,0,0,0,23,0,0
Python,0,0,0,8,19,0
Ruby,0,0,0,1,0,114


In [99]:
val_pred = clf.predict(X_val)

In [100]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_val, y_val)))

Accuracy of Decision Tree classifier on training set: 0.82


In [101]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=5,
                            n_estimators=250,
                            max_depth=6, 
                            random_state=77)

In [102]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=5, n_estimators=250,
                       random_state=77)

In [103]:
rf_pred = rf.predict(X_train)

In [104]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.84


In [78]:
rf2_pred = rf.predict(X_val)

In [79]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_val, y_val)))

Accuracy of random forest classifier on training set: 0.67


In [106]:
df2.language.value_counts()

Ruby          213
Other          38
Python         36
Java           36
JavaScript     35
C_based        22
Name: language, dtype: int64

In [109]:
len(df2[df2['language'] == 'Ruby'])/len(df2)

0.5605263157894737